# Notebook to prep functions for report

In [1]:
import intake
import numpy as np
import pandas as pd
from calitp import to_snakecase
#from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *

import altair as alt

import _data_cleaning
import _report_utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


ImportError: cannot import name 'tbl' from 'calitp.tables' (/opt/conda/lib/python3.10/site-packages/calitp/tables.py)

In [ ]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [ ]:
df = _report_utils.read_in_joined_data()

In [ ]:
pd.set_option("display.max_columns",500)

In [ ]:
df.head()

In [ ]:
#check to make sure this all worked - should have some Awarded Yes
(df>>arrange(_.project_app_id, _.project_cycle)>>select(_.project_app_id, _.project_cycle, _.awarded, _.data_origin)).head(50)

In [ ]:
df>>group_by(_.project_cycle)>>count(_.data_origin)

In [ ]:
df>>group_by(_.project_cycle, _.awarded)>>count(_.a2_county)>>filter(_.project_cycle==5, _.n>0)>>arrange(_.a2_county)

## ISSUE

* county name is a code in the funded data. need to convert

In [ ]:
county_place_names = (to_snakecase(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/2020-place-names-locode.xlsx', sheet_name=1)))

In [ ]:
county_place_names = county_place_names>>select(_.county_name, _.co__name_abbr_)

In [ ]:
county_place_names['co__name_abbr_'] = county_place_names['co__name_abbr_'].str.upper()

In [ ]:
county_map = dict(county_place_names[['co__name_abbr_', 'county_name']].values)

In [ ]:
county_map

In [ ]:
#need to make a new column, then fill na with og column
#df['a2_county_2'] = df.a2_county.map(county_map)

In [ ]:
df>>group_by(_.project_cycle, _.awarded)>>count(_.a2_county)>>filter(_.project_cycle==5, _.n>0)>>arrange(_.a2_county)